In [ ]:

from nifty import *

"""
define some signal space for example: 
a. 1D regular grid
b. 2D regular grid

usually the conjugate space should also be defined.
"""

#x_space= rg_space(200)          # 1D signal space
x_space= rg_space([200,200])    # 2D signal space

k_space= x_space.get_codomain()

"""
creating data according to d = Rs + n 

to do so: 
1. define a Reponse operator
2. define Signal covariance S 
3. define Noise covariance N
4. get random fields drawn from the covariances for the test data set
5. get the data d
"""

R= response_operator(x_space, sigma=0.0, mask=1.0, assign=None)
d_space=R.target

np.random.seed(42)                          #set the seed for random number generator

power_S= (lambda k: 42/(1+k)**4)            #define spectrum of Signal covariance
S= power_operator(k_space, spec= power_S)   # define Signal covariance operator
s= S.get_random_field(domain=x_space)       # get a random s according to S

N= diagonal_operator(d_space, diag=s.var(), bare=True)  #define Noise covariance
n= N.get_random_field(domain=d_space)                   # get some noise

d= R(s)+n               #generate the data for the following analysis

class Wiener_problem(object):
    
    """
        this class solve the Wiener filter problem in various ways:
         1. m= Dj, in case R, S, N are known 
         2. running a critical filter, i.e. only R and N are known,
            S is inferred from the data
    
    Parameters:
        a. domain of the signal field (default: x_space)
        b. the codomain of the signal field (default: k_space)
        c. the raw data (default: d)
        d. the response operator (default: R)
        e. the Signal_covariance (default: S)
        f. the noise covariance (default: N)
        
    Funtions: 
    
    Conjugate gradient:
    solve the Wiener filter problem by using the conjugate gradient method to find m
    
    Hamilton_gradient: 
    evaluates the Hamiltonian and its gradient given a field x. 
    
    Steepest_descent:
    runs an iterative downhill algorithm to find the minimum of the Hamiltonian
    
        
    """
    
    def __init__(self, domain=x_space, codomain=k_space, data=d, 
                 Response_operator=R, Signal_covariance=S, 
                 Noise_covariance=N):
        
        if(domain is None):
            raise TypeError ("Missing signal space")
        else: 
            self.domain= domain
        
        if(codomain is None):
            raise TypeError ("Missing codomain of the signal space")
        else: 
            self.codomain=codomain
        
        if(data is None):
            raise TypeError ("Missing data")
        else: 
            self.d= data
        
        if (Response_operator is None):
            raise TypeError ("Missing Response operator")
        else:
            self.R=Response_operator
            
        if(Signal_covariance is None):
            raise TypeError ("Missing signal covariance")
        else:
            self.S= Signal_covariance
        
        if(Noise_covariance is None):
            raise TypeError ("Missing noise covariance")
        else: 
            self.N= Noise_covariance
            
        self.D= propagator_operator(S= self.S, N=self.N, R=self.R)
        self.j= self.R.adjoint_times(self.N.inverse_times(self.d))
        
    def conjugate_gradient(self):
        m = self.D.times(self.j, W=self.S, note=True, clevel=3, tol=1E-3) 
#        err_m= self.D.hat()
#        m.plot(title="reconstructed signal")
#        err_m.plot(title="uncertainty map")
#        s.plot(title="signal")
#        field(x_space, val=d.val).plot(title="analyzed data")
        return m
                
    def Hamiltonian_gradient(self, x):
        """
            Calculation of the information hamiltonian and its gradient
        """
        DIx= self.D.inverse_times(x)        
        H= 0.5* DIx.dot(x)-self.j.dot(x)
        g= DIx-self.j
        return H, g
        
    def steepest_descent(self):
        m= field(x_space, target= k_space)
        m, convergence= steepest_descent(eggs= self.Hamiltonian_gradient, note=True)(m, clevel=4, tol=1E-4)
        return m
        
    
        
                     
                     